> 运行前请安装dhg: `pip install git+https://github.com/iMoonLab/DeepHypergraph.git`

In [17]:
import sys

sys.path.append("../")  # 添加项目根目录到路径中

In [18]:
import torch
import torch.optim as optim
from torch import nn
torch.manual_seed(42)

import pickle

import dhg
from dhg import Hypergraph

import hgp
from hgp.models import HGNNP
from hgp.loss import loss_bs_matrix
from hgp.utils import from_file_to_hypergraph
from hgp.function import StraightThroughEstimator

DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE

device(type='cuda', index=1)

In [19]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 3

"""
h_hyper_prmts["convlayers1"] = {"in_channels": 3824, "out_channels": 1024, "use_bn": False, "drop_rate": 0.4}
h_hyper_prmts["convlayers3"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers4"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers5"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.2}

l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":128, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer2"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.1}
l_hyper_prmts["linerlayer3"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.1}
l_hyper_prmts["linerlayer4"] = {"in_channels":32, "out_channels":3, "use_bn":False, "drop_rate":0.1}
"""
"""
h_hyper_prmts["convlayers1"] = {"in_channels": 4096, "out_channels": 2048, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers11"] = {"in_channels": 2048, "out_channels": 2048, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers12"] = {"in_channels": 2048, "out_channels": 1024, "use_bn": False, "drop_rate": 0.3}
#h_hyper_prmts["convlayers13"] = {"in_channels": 2048, "out_channels": 2048, "use_bn": False, "drop_rate": 0.3}
#h_hyper_prmts["convlayers14"] = {"in_channels": 2048, "out_channels": 1024, "use_bn": False, "drop_rate": 0.25}
#h_hyper_prmts["convlayers14"] = {"in_channels": 1536, "out_channels": 1536, "use_bn": False, "drop_rate": 0.3}
#h_hyper_prmts["convlayers15"] = {"in_channels": 1536, "out_channels": 1536, "use_bn": False, "drop_rate": 0.3}
#h_hyper_prmts["convlayers16"] = {"in_channels": 2048, "out_channels": 1536, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers3"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers4"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers5"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.2}
#h_hyper_prmts["convlayers52"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.1}


l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":1024, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer2"] = {"in_channels":1024, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer21"] = {"in_channels":512, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer22"] = {"in_channels":512, "out_channels":256, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer23"] = {"in_channels":256, "out_channels":128, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer3"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer32"] = {"in_channels":64, "out_channels":32, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer33"] = {"in_channels":32, "out_channels":16, "use_bn":False, "drop_rate":0.05}
l_hyper_prmts["linerlayer4"] = {"in_channels":16, "out_channels":3, "use_bn":False, "drop_rate":0.05}
"""
h_hyper_prmts["convlayers14"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers1"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers12"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.3}
h_hyper_prmts["convlayers13"] = {"in_channels": 1024, "out_channels": 512, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers3"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers4"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers5"] = {"in_channels": 512, "out_channels": 512, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers52"] = {"in_channels": 512, "out_channels": 256, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers53"] = {"in_channels": 256, "out_channels": 128, "use_bn": False, "drop_rate": 0.2}


l_hyper_prmts["linerlayer1"] = {"in_channels":list(h_hyper_prmts.values())[-1]["out_channels"], "out_channels":128, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer4"] = {"in_channels":128, "out_channels":3, "use_bn":False, "drop_rate":0.05}

hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [20]:
def loss_label_loss(outs,label,device):
    # fmt: off
    r"""
    对网络进行预训练的代码
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``label``(`Hypergraph`):  vertex的标签. Size :math:`(N, nums_classes)`. 
    """
    # fmt: on
    
    

In [21]:
def loss_bs_matrix(outs, hg, device):
    # fmt: off
    r"""
    对于超图的损失函数的矩阵形式.
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``hg``(`Hypergraph`):  超图对象.  
    """
    # fmt: on
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree -0.35

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)

    loss = 50*loss_1 + loss_2
    return loss, loss_1, loss_2

#### 定义用于训练的类Trainer

In [22]:
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.  
    """
    # fmt: on
    def __init__(self, net, X, hg, optimizer,label):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.label = label.to(DEVICE)
        
    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def pre_train(self, epoch):
        self.train()
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss = nn.CrossEntropyLoss()(outs, self.label)
        loss.backward()
        self.optimizer.step()
        return loss.item()
    
    def run(self, epoch):
        self.train()  # train mode | 设置为训练模式
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(outs, self.hg, device=DEVICE)
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [23]:
G = from_file_to_hypergraph("../data/ISPD_benchmark/ibm01_new.hgr",reset_vertex_index=True)
edges, num_v = G.e,G.num_v
with open("../data/pretrain/label_ibm01.pkl", "rb") as f:
    label = pickle.load(f)
G.num_e,G.num_v,len(label)

(13257, 12752, 12752)

In [24]:
X = torch.randn(size=(num_v, hyper["init_features_dim"]))
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None, label=label).to(DEVICE)

for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(nn.ReLU().to(DEVICE))
    hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))

optim = optim.Adam(hgnn_trainer.parameters(), lr=4e-4, weight_decay=0)
hgnn_trainer.optimizer = optim

RuntimeError: size is inconsistent with indices: for dim 0, size is 12752 but found index 12752

In [ ]:
temp_loss_total= torch.zeros(1, requires_grad=False)
for epoch in range(200):
    loss = hgnn_trainer.pre_train(epoch=epoch)
    # train
    temp_loss_total += loss
    # validation
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        sys.stdout.flush()
        temp_loss_total = torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 0.11060378551483155


in 10 epoch, average loss: 0.8473074913024903
in 20 epoch, average loss: 0.6908018589019775
in 30 epoch, average loss: 0.6286892414093017
in 40 epoch, average loss: 0.5772213935852051
in 50 epoch, average loss: 0.5300322055816651
in 60 epoch, average loss: 0.4841326713562012
in 70 epoch, average loss: 0.43969430923461916
in 80 epoch, average loss: 0.4011239051818848
in 90 epoch, average loss: 0.3679665565490723
in 100 epoch, average loss: 0.339047908782959
in 110 epoch, average loss: 0.3145130157470703
in 120 epoch, average loss: 0.29218101501464844
in 130 epoch, average loss: 0.2736319303512573
in 140 epoch, average loss: 0.2569025754928589
in 150 epoch, average loss: 0.2419965982437134
in 160 epoch, average loss: 0.22755489349365235
in 170 epoch, average loss: 0.21446738243103028
in 180 epoch, average loss: 0.20234029293060302
in 190 epoch, average loss: 0.19116994142532348


In [ ]:
hgnn_trainer.forward(hgnn_trainer.X)

tensor([[-2.6352,  1.3473, -3.0027],
        [-2.6518,  1.3088, -2.9908],
        [-2.0291,  1.5059, -2.4597],
        ...,
        [-2.5938,  1.3281, -2.9586],
        [-2.2752,  1.1233, -2.5182],
        [-2.4658,  1.3350, -2.8108]], device='cuda:1',
       grad_fn=<AddmmBackward0>)

In [ ]:
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.3, inplace=False)
        (theta): Linear(in_features=2048, out_features=1024, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.3, inplace=False)
        (theta): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.3, inplace=False)
        (theta): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (3): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=1024, out_features=512, bias=True)
      )
      (4): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=512, out_features=512, bias=True)
      )
    )
  )
  (1): BatchNorm1d(512, eps=1e-

In [ ]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
for epoch in range(15000):
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    # train
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    # validation
    if epoch % 1 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 1}")
        print(f"                , loss1: {temp_loss1.item() / 1}")
        print(f"                , loss2: {temp_loss2.item() / 1}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 973518.9375
                , loss1: -6697.68603515625
                , loss2: 1308403.25
in 1 epoch, average loss: 773803.6875
                , loss1: -6405.81640625
                , loss2: 1094094.5
in 2 epoch, average loss: 973464.375
                , loss1: -4559.97998046875
                , loss2: 1201463.375
in 3 epoch, average loss: 273287.4375
                , loss1: -5463.208984375
                , loss2: 546447.875
in 4 epoch, average loss: 94962.015625
                , loss1: -4428.5078125
                , loss2: 316387.40625
in 5 epoch, average loss: -184372.90625
                , loss1: -4392.8701171875
                , loss2: 35270.5859375
in 6 epoch, average loss: -190836.546875
                , loss1: -3857.427734375
                , loss2: 2034.845458984375
in 7 epoch, average loss: -146988.375
                , loss1: -3513.95654296875
                , loss2: 28709.4453125
in 8 epoch, average loss: -180610.453125
               

KeyboardInterrupt: 

In [ ]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

In [ ]:
bs = torch.sum(outs_straight, dim = 0)
# bs = torch.sum(outs, dim = 0)
bs